In [1]:
from pyspark.sql import SparkSession
import math

In [2]:
from pyspark.sql.functions import variance, stddev_samp

In [3]:
spark = SparkSession.builder.appName("GNB_Solution").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

21/08/02 10:31:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [5]:
dataSet = spark.read.csv("naive.csv", inferSchema=True, header=True)

In [6]:
dataSet.show()

+---+------+---------+
|Age|Salary|Purchased|
+---+------+---------+
| 59| 88000|        1|
| 35| 61000|        0|
| 37| 70000|        1|
| 52| 21000|        1|
| 48|141000|        0|
| 37| 93000|        1|
| 37| 62000|        0|
| 48|138000|        1|
| 41| 79000|        0|
| 37| 78000|        1|
| 39|134000|        1|
| 49| 89000|        1|
| 55| 39000|        1|
| 37| 77000|        0|
| 35| 57000|        0|
| 36| 63000|        0|
| 42| 73000|        1|
+---+------+---------+



In [40]:
means_Salary = dataSet.groupBy("Purchased").mean("Salary")
means_Salary.show()

+---------+-----------------+
|Purchased|      avg(Salary)|
+---------+-----------------+
|        1|          82300.0|
|        0|77142.85714285714|
+---------+-----------------+



In [38]:
means_age = dataSet.groupBy("Purchased").mean("Age")
means_age.show()

+---------+-----------------+
|Purchased|         avg(Age)|
+---------+-----------------+
|        1|             45.5|
|        0|38.42857142857143|
+---------+-----------------+



In [39]:
stdv_Salary = dataSet.groupBy("Purchased").agg(stddev_samp("Salary"))
stdv_Salary.show()

+---------+-------------------+
|Purchased|stddev_samp(Salary)|
+---------+-------------------+
|        1| 36295.239969511764|
|        0|  29373.61952176687|
+---------+-------------------+



In [26]:
stdv_age = dataSet.groupBy("Purchased").agg(stddev_samp("Age"))
stdv_age.show()

+---------+-----------------+
|Purchased| stddev_samp(Age)|
+---------+-----------------+
|        1|8.195527235429498|
|        0| 4.68533680244878|
+---------+-----------------+



In [27]:
variances_salary = dataSet.groupBy("Purchased").agg(variance("Salary"))
variances_salary.show()

+---------+--------------------+
|Purchased|    var_samp(Salary)|
+---------+--------------------+
|        1|1.3173444444444444E9|
|        0| 8.628095238095237E8|
+---------+--------------------+



In [28]:
variances_age = dataSet.groupBy("Purchased").agg(variance("Salary"))
variances_age.show()

+---------+--------------------+
|Purchased|    var_samp(Salary)|
+---------+--------------------+
|        1|1.3173444444444444E9|
|        0| 8.628095238095237E8|
+---------+--------------------+



In [30]:
dataset_yes = dataSet.filter("Purchased = True")
dataset_yes.show()

+---+------+---------+
|Age|Salary|Purchased|
+---+------+---------+
| 59| 88000|        1|
| 37| 70000|        1|
| 52| 21000|        1|
| 37| 93000|        1|
| 48|138000|        1|
| 37| 78000|        1|
| 39|134000|        1|
| 49| 89000|        1|
| 55| 39000|        1|
| 42| 73000|        1|
+---+------+---------+



In [31]:
dataset_no = dataSet.filter("Purchased = False")
dataset_no.show()

+---+------+---------+
|Age|Salary|Purchased|
+---+------+---------+
| 35| 61000|        0|
| 48|141000|        0|
| 37| 62000|        0|
| 41| 79000|        0|
| 37| 77000|        0|
| 35| 57000|        0|
| 36| 63000|        0|
+---+------+---------+



In [32]:
n_total = dataSet.count()

In [33]:
n_yes = dataset_yes.count()

In [34]:
n_no = dataset_no.count()

In [35]:
p_yes = n_yes / n_total

In [36]:
p_no = n_no / n_total

In [37]:
print(p_yes)
print(p_no)

0.5882352941176471
0.4117647058823529


In [17]:
dataSet.groupBy("PassExam").mean("Score").collect()

[Row(PassExam='No', avg(Score)=29.8),
 Row(PassExam='Yes', avg(Score)=76.16666666666667)]

In [18]:
mean_n = dataSet.groupBy("PassExam").mean("Score").collect()[0][1]

In [19]:
mean_y = dataSet.groupBy("PassExam").mean("Score").collect()[1][1]

In [20]:
print(mean_n)
print(mean_y)

29.8
76.16666666666667


In [21]:
variance_n = dataSet.groupBy("PassExam").agg(variance("Score")).collect()[0][1]

In [22]:
variance_y = dataSet.groupBy("PassExam").agg(variance("Score")).collect()[1][1]

In [23]:
print(variance_n)
print(variance_y)

131.70000000000002
267.76666666666665


In [31]:
final_p_y = ((1 / math.sqrt(2 * math.pi * variance_y)) * math.exp((-0.5 * ((61 - mean_y) ** 2) / variance_y)))

In [32]:
final_p_n = ((1 / math.sqrt(2 * math.pi * variance_n)) * math.exp((-0.5 * ((61 - mean_n) ** 2) / variance_n)))

### P(61|Yes)

In [33]:
print(final_p_y)

0.01586679399696767


## P(61|No)

In [34]:
print(final_p_n)

0.0008631916482906215


### Likelihood P(Yes|61) & P(NO|61)

In [35]:
likelihood_y = p_yes * final_p_y
likelihood_n = p_no * final_p_n

In [36]:
print(f'Likelihood Yes: {likelihood_y}')
print(f'Likelihood NO: {likelihood_n}')

Likelihood Yes: 0.00865461490743691
Likelihood NO: 0.00039235984013210066


In [37]:
print("Yes" if likelihood_y > likelihood_n else "No")

Yes
